In [1]:
import tensorflow as tf
import utils
import pandas as pd
import numpy as np
__author__ = "Olivares Castillo José Luis"

tf.__version__

'1.5.0-rc0'

In [2]:
es,na = utils.load_node2vec()

In [3]:
na_dummy = na.drop(na.columns[0],axis=1)
na_vectores1 = np.array(na_dummy)

In [4]:
eval_set = pd.read_csv("../lexiconevaluacion/evaluationset",delimiter=" ",names=["esp","nah"])

In [5]:
eval_set.head()

,esp,nah
0,traer,itqui
1,traer,huica
2,traer,huiqui
3,traer,cui
4,adornar,apana


In [6]:
eval_es = list(set(eval_set["esp"]))

In [7]:
eval_es

['visceras',
 'rostro',
 'dar',
 'comer',
 'tepaneca',
 'perdonar',
 'pesar',
 'coyote',
 'tierno',
 'salvaje',
 'saltar',
 'sueño',
 'pedirle',
 'olvidar',
 'tortuga',
 'echar',
 'fino',
 'agregar',
 'galleta',
 'reinar',
 'izquierda',
 'tigre',
 'sólo',
 'conocer',
 'aguacate',
 'diverso',
 'mirada',
 'tomar',
 'pantalón',
 'limpiar',
 'tanto',
 'cultura',
 'naranja',
 'rogar',
 'seno',
 'establecer',
 'querer',
 'pez',
 'pasar',
 'llevar',
 'anáhuac',
 'prestar',
 'jadear',
 'ramo',
 'lodo',
 'patio',
 'brazo',
 'leche',
 'tapar',
 'muerto',
 'valer',
 'aire',
 'indígena',
 'descanso',
 'pídele',
 'suceder',
 'barca',
 'señorito',
 'pluma',
 'montaña',
 'grande',
 'traer',
 'centenario',
 'adquisición',
 'pipián',
 'quetzal',
 'unir',
 'efecto',
 'paso',
 'perforar',
 'jugar',
 'adornar',
 'salvador',
 'vasija',
 'admirar',
 'volver',
 'voz',
 'decir',
 'ropa',
 'nabo',
 'madera',
 'alegrar',
 'transportar',
 'pieza',
 'presidente',
 'acabar',
 'esfuerzo',
 'amado',
 'caja',
 'escal

In [8]:
eval_es_index = [int(es[es[0] == palabra].index[0])
                  for palabra in eval_es]

In [9]:
es[es[0] == "pluma"]

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
2902,pluma,-0.386967000000000005,0.005026,-0.191887000000000002,-0.081058000000000005,-0.211675000000000002,-0.217542999999999986,-0.019026000000000001,0.454046000000000005,0.006309,...,-0.013390000000000001,0.023695999999999998,0.394602000000000008,-0.003221,-0.708667000000000047,0.021819000000000002,0.181774999999999992,0.436242000000000019,-0.162193000000000004,-0.076617000000000005


In [10]:
eval_es_index[0]

2943

In [11]:
eval_es_vectores = utils.get_vectors(es,eval_es_index)

In [12]:
eval_es_vectores.shape

(150, 128)

In [13]:
test_vectors = np.array([np.array(es.iloc[indice][1::]).astype(np.float64) for indice in eval_es_index])

In [14]:
test_vectors[0]

array([-0.351641, -0.159605,  0.196677, -0.158129,  0.087288, -0.114435,
       -0.276387, -0.319244, -0.122458, -0.128152,  0.026067,  0.018235,
       -0.028133, -0.021822,  0.055507,  0.06958 ,  0.434447,  0.229957,
       -0.250883, -0.233929,  0.02432 , -0.151442,  0.531101,  0.399131,
        0.568763, -0.075973,  0.042421,  0.184531, -0.195088, -0.163787,
        0.032918, -0.242526, -0.179619, -0.145038,  0.009259,  0.13187 ,
        0.305791, -0.26803 ,  0.267837,  0.460017,  0.006644,  0.291476,
       -0.058206, -0.114762, -0.037404,  0.225061,  0.14716 , -0.448858,
       -0.21894 ,  0.127921,  0.116462,  0.379808, -0.253464,  0.042928,
       -0.248884,  0.571844,  0.220163, -0.467988,  0.220994, -0.091717,
       -0.24746 , -0.334188,  0.026831, -0.482147,  0.202073,  0.121547,
       -0.653929, -0.045984, -0.079111, -0.288587, -0.156731,  0.437338,
        0.281268, -0.13055 ,  0.019042, -0.004954,  0.137847,  0.175403,
        0.170409,  0.46675 , -0.154581,  0.162832, 

In [15]:
sess = tf.Session()

In [16]:
saver = tf.train.import_meta_graph('./models/model2254_gpu/model2254.ckpt.meta')
saver.restore(sess,tf.train.latest_checkpoint('./models/model2254_gpu/'))

INFO:tensorflow:Restoring parameters from ./models/model2254_gpu/model2254.ckpt


In [17]:
graph = tf.get_default_graph()

In [18]:
X = graph.get_tensor_by_name("input/input_es:0")
#y = graph.get_tensor_by_name("input/target_na:0")

In [19]:
output_NN = graph.get_tensor_by_name("output/xw_plus_b:0")

In [20]:
feed_dict = {X:test_vectors}


In [21]:
pred = sess.run(output_NN,feed_dict)
print (type(pred[0]),pred.shape)

<class 'numpy.ndarray'> (150, 128)


In [22]:
top_10=[utils.get_top10_closest(pred[_],na_vectores1) for _ in range(pred.shape[0])]

In [23]:
closest = [utils.get_closest_words_to(top_10[_],na) for _ in range(pred.shape[0])]

In [24]:
es[es[0]=="adquisición"].index

Int64Index([2222], dtype='int64')

In [25]:
na[na[0]=="v"].index

Int64Index([1376], dtype='int64')

In [26]:
eval_es[6]

'pesar'

In [27]:
closest[6]

['palehui',
 'tepahpaquiltia',
 'xip',
 'xihui',
 'ticcua',
 'dral',
 'diego',
 'secretario',
 'yb',
 'rosario']

In [28]:
resultados = {palabra_es:top_10_nah for (palabra_es,top_10_nah) in zip(eval_es,closest)}

In [29]:
resultados

{'acabar': ['mic',
  'cauh',
  'tlami',
  'auh',
  'ahui',
  'miqui',
  'tia',
  'xip',
  'tami',
  'tepahpaquiltia'],
 'admirar': ['mahuizo',
  'imottilih',
  'onimitz',
  'chililiz',
  'itt',
  'mihtlanililia',
  'mahui',
  'xip',
  'itti',
  'zcuco'],
 'adornar': ['uan',
  'chihchihua',
  'dral',
  'rosario',
  'secretario',
  'diego',
  'castillo',
  'oztomeca',
  'acamapich',
  'tepahpaquiltia'],
 'adquisición': ['oticma',
  'chiquiuh',
  'mel',
  'amech',
  'lahtol',
  'amaneapan',
  'ceuh',
  'immoma',
  'ommihzo',
  'yb'],
 'agregar': ['motlali',
  'tiliaya',
  'tlahuicxitilli',
  'tec',
  'cuacualaca',
  'tlalia',
  'huic',
  'caxi',
  'izta',
  'itzoltilia'],
 'aguacate': ['aztlacapalli',
  'tolachno',
  'ahua',
  'poyox',
  'pix',
  'diego',
  'dral',
  'rosario',
  'secretario',
  'castillo'],
 'aire': ['eheca',
  'yehyecatl',
  'tetzahui',
  'huilo',
  'yb',
  'dral',
  'diego',
  'xip',
  'eca',
  'secretario'],
 'alegrar': ['tlaza',
  'tehhuellamac',
  'pahpaqui',
  'cui

In [30]:
esp = list(eval_set["esp"].values)

In [31]:
nah = list(eval_set["nah"].values)

In [32]:
pares_eval = list(zip(esp,nah))

In [33]:
from collections import defaultdict

In [34]:
gold = defaultdict(list)
for palabra_es,palabra_na in pares_eval:
    gold[palabra_es].append(palabra_na)


In [35]:
gold = dict(gold)

In [36]:
gold

{'acabar': ['tlami', 'tami'],
 'admirar': ['mahuizo', 'iglesia', 'tlamahuiz'],
 'adornar': ['apana', 'chihchihua'],
 'adquisición': ['quipiaz', 'tocococauh', 'cococauh', 'cococa'],
 'agregar': ['motlali', 'tlalia'],
 'aguacate': ['ahua'],
 'aire': ['yehyecatl', 'yeyeca', 'eheca', 'eheca'],
 'alegrar': ['ahuia', 'pahpa', 'pahpaqui'],
 'alfredo': ['alfredo'],
 'alimento': ['pal', 'atetzocoa'],
 'amado': ['tlazo', 'tlazohtla'],
 'amargo': ['chichi'],
 'animal': ['yolca', 'yoyolcameh'],
 'anáhuac': ['nahuac'],
 'barca': ['acalli'],
 'brazo': ['aca'],
 'caja': ['petlacal'],
 'cargo': ['tlahtoca', 'toca'],
 'centenario': ['nario', 'presi'],
 'chico': ['notelpochtli', 'tzitz', 'tepi'],
 'chiquillo': ['pipil'],
 'cierto': ['melahua', 'nel', 'cecni'],
 'ciudad': ['altpe', 'altepe'],
 'cofre': ['top', 'petlacal'],
 'coger': ['ana', 'cana', 'cui'],
 'comenzar': ['pehua', 'peuh'],
 'comer': ['cuaz', 'cua', 'tlacual', 'otlacua', 'cualtiz'],
 'conocer': ['ixmati', 'ixmachilia', 'mati', 'oixmatilia',

In [37]:
gold["amado"]

['tlazo', 'tlazohtla']

In [38]:
resultados["amado"]

['tlazohtla',
 'ama',
 'zquia',
 'ticcua',
 'diego',
 'comal',
 'federal',
 'tlazo',
 'secretario',
 'dral']

In [39]:
resultados["aguacate"][1:5]

['tolachno', 'ahua', 'poyox', 'pix']

In [40]:
gold["comer"][1] in resultados["comer"][0]

True

In [41]:
resultados["aguacate"][0:1]

['aztlacapalli']

In [42]:
list(resultados.keys())

['visceras',
 'rostro',
 'dar',
 'comer',
 'tepaneca',
 'perdonar',
 'pesar',
 'coyote',
 'tierno',
 'salvaje',
 'saltar',
 'sueño',
 'pedirle',
 'olvidar',
 'tortuga',
 'echar',
 'fino',
 'agregar',
 'galleta',
 'reinar',
 'izquierda',
 'tigre',
 'sólo',
 'conocer',
 'aguacate',
 'diverso',
 'mirada',
 'tomar',
 'pantalón',
 'limpiar',
 'tanto',
 'cultura',
 'naranja',
 'rogar',
 'seno',
 'establecer',
 'querer',
 'pez',
 'pasar',
 'llevar',
 'anáhuac',
 'prestar',
 'jadear',
 'ramo',
 'lodo',
 'patio',
 'brazo',
 'leche',
 'tapar',
 'muerto',
 'valer',
 'aire',
 'indígena',
 'descanso',
 'pídele',
 'suceder',
 'barca',
 'señorito',
 'pluma',
 'montaña',
 'grande',
 'traer',
 'centenario',
 'adquisición',
 'pipián',
 'quetzal',
 'unir',
 'efecto',
 'paso',
 'perforar',
 'jugar',
 'adornar',
 'salvador',
 'vasija',
 'admirar',
 'volver',
 'voz',
 'decir',
 'ropa',
 'nabo',
 'madera',
 'alegrar',
 'transportar',
 'pieza',
 'presidente',
 'acabar',
 'esfuerzo',
 'amado',
 'caja',
 'escal

In [43]:
len(list(resultados.keys()))

150

In [44]:
p1 = 0
p5 = 0
p10 = 0
f1,f5,f10=False,False,False
palabra_gold = "madera"
hits=list()

not_found = 0

for palabra_gold in list(resultados.keys()):
    for i in gold[palabra_gold]:
        if i in resultados[palabra_gold]:
            hits.append(resultados[palabra_gold].index(i))
    if hits.__len__() > 0:
        if min(hits) == 0:
            p1 += 1
            p5 += 1
            p10 += 1
        if min(hits) >= 1 and min(hits) <= 5:
            p5 += 1
            p10 += 1
        if min(hits) > 5 and min(hits) <= 10:
            p10 += 1
        print(palabra_gold,min(hits),hits,p1,p5,p10)
    else:
        not_found += 1
        print(palabra_gold+": NOT FOUND")
    
    
    
        
    hits.clear()

print("P@1:",p1/len(list(resultados.keys())),"\tP@5:",p5/len(list(resultados.keys())),
      "\tP@10:",p10/len(list(resultados.keys())))



visceras: NOT FOUND
rostro 0 [0] 1 1 1
dar 0 [2, 0] 2 2 2
comer 0 [4, 0, 2] 3 3 3
tepaneca 2 [2] 3 4 4
perdonar 0 [0] 4 5 5
pesar 4 [4] 4 6 6
coyote 0 [0] 5 7 7
tierno 0 [0] 6 8 8
salvaje 0 [0] 7 9 9
saltar 1 [1] 7 10 10
sueño 0 [0] 8 11 11
pedirle 0 [0] 9 12 12
olvidar 2 [2] 9 13 13
tortuga: NOT FOUND
echar 1 [1] 9 14 14
fino 3 [5, 3] 9 15 15
agregar 0 [0, 5] 10 16 16
galleta 0 [0] 11 17 17
reinar: NOT FOUND
izquierda 0 [0] 12 18 18
tigre 0 [0, 2] 13 19 19
sólo 0 [0] 14 20 20
conocer 0 [0, 2, 5] 15 21 21
aguacate 2 [2] 15 22 22
diverso 0 [0] 16 23 23
mirada 0 [0] 17 24 24
tomar 8 [8] 17 24 25
pantalón 0 [0] 18 25 26
limpiar 0 [1, 0] 19 26 27
tanto 5 [5] 19 27 28
cultura 0 [1, 0] 20 28 29
naranja 0 [0] 21 29 30
rogar 1 [1] 21 30 31
seno 0 [0] 22 31 32
establecer: NOT FOUND
querer 0 [1, 0, 9] 23 32 33
pez 0 [0] 24 33 34
pasar 0 [0] 25 34 35
llevar 0 [0, 1] 26 35 36
anáhuac 0 [0] 27 36 37
prestar 0 [0] 28 37 38
jadear 0 [0] 29 38 39
ramo 0 [0] 30 39 40
lodo 1 [1] 30 40 41
patio 0 [0, 1, 

In [45]:
not_found

22